In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json
import contextily as ctx
import warnings
import numpy as np

warnings.filterwarnings("ignore")

import json
def load_json(filename):
    with open(filename, 'r') as f:
        json_data = json.load(f)
    return json_data

# Carga de los datos originales de la grilla
def load_original_grid():
    grid_filename = '/app/data/input/grid_cityscope.geojson'  
    return load_json(grid_filename)

def convert_list_columns(gdf, list_of_columns):
    for col in list_of_columns:
        gdf[col] = gdf[col].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
        gdf[col] = gdf[col].apply(json.dumps)
    return gdf

In [2]:
grid_original = load_original_grid()

# Extraccion de las propiedades que son estaticas (numero de filas, columnas, proyección, etc)
grid_header = grid_original['GEOGRID']['properties']['header']

In [3]:
# Creación de la nueva grilla para cargarla
grid_output = {}
grid_output['GEOGRID'] = {}
grid_output['GEOGRIDDATA'] = {}
grid_output['GEOGRID']['properties'] = {}
grid_output['GEOGRID']['properties']['header'] = grid_header
grid_output['GEOGRID']['type'] = 'FeatureCollection'

In [4]:
grid = gpd.read_parquet('/app/data/landuses_grid.parquet')
# Convertir arrays a strings
list_of_columns = ['color', 'height']
grid = convert_list_columns(grid, list_of_columns)

In [5]:
GEOGRID = grid[['id', 'interactive', 'name', 'height', 'color', 'geometry']]
GEOGRID.to_file('/app/data/GEOGRID.geojson', driver='GeoJSON')

In [6]:
GEOGRIDDATA = GEOGRID.drop(columns=['geometry'])
GEOGRIDDATA.to_json('/app/data/GEOGRIDDATA.json', orient='records')

In [7]:
# Salida del geojson final, etapa 1 de pruebas
output_filename = '/app/data/testing_1.geojson'

with open(output_filename, 'w') as archivo:
    json.dump(grid_output, archivo)

### Agregar codigos de LBCS a los types

In [8]:
LBCS = {
    'SITIO ERIAZO': 9000,
    'BIENES COMUNES': 9999,
    'OTROS': 9200,
    'SALUD': 6500,
    'CULTO': 6600,
    'BODEGA Y ALMACENAJE': 3120,
    'HABITACIONAL': 1100,
    'OFICINA': 2300,
    'COMERCIO': 2100,
    'EDUCACION Y CULTURA': 4100,
    'INDUSTRIA': 3000,
    'AREAS VERDES': 7000,
    'PRE ROL': 9999,
    'TRANSPORTE Y TELEC': 4000, #434public0 Telecomunicaciones
    'DEPORTE Y RECREACION': 7100,
    'HOTEL, MOTEL': 1200,
    'ESTACIONAMIENTO': 5210,
    'ADM PUBLICA Y DEFENSA': 6200, #+6300
    'MULTI ROL': 9999,
    'AGRICOLA': 9000,
    'S/I': 9999,
}

LBCS_translated = {
    'SITIO ERIAZO': 'Vacant lot',
    'BIENES COMUNES': 'Public spaces',
    'OTROS': 'Other',
    'SALUD': 'Healthcare',
    'CULTO': 'Religious',
    'BODEGA Y ALMACENAJE': 'Warehouse and storage',
    'HABITACIONAL': 'Residential',
    'OFICINA': 'Office',
    'COMERCIO': 'Commercial',
    'EDUCACION Y CULTURA': 'Education and culture',
    'INDUSTRIA': 'Industry',
    'AREAS VERDES': 'Park',
    'TRANSPORTE Y TELEC': 'Transportation and telecommunications', #4340 Telecommunications
    'DEPORTE Y RECREACION': 'Sports and recreation',
    'HOTEL, MOTEL': 'Hotel, motel',
    'ESTACIONAMIENTO': 'Parking',
    'ADM PUBLICA Y DEFENSA': 'Public administration and defense', #+6300
    'AGRICOLA': 'Agricultural',
    'S/I': 'None'
}

color_dict = {
    'SITIO ERIAZO': '#E57373',
    'HABITACIONAL': '#81C784',
    'EDUCACION Y CULTURA': '#64B5F6',
    'BODEGA Y ALMACENAJE': '#F06292',
    'BIENES COMUNES': '#DCE775',
    'OTROS': '#4DD0E1',
    'INDUSTRIA': '#BA68C8',
    'SALUD': '#FFB74D',
    'CULTO': '#9575CD',
    'OFICINA': '#AED581',
    'COMERCIO': '#F48FB1',
    'TRANSPORTE Y TELEC': '#B0BEC5',
    'AREAS VERDES': '#4CAF50',
    'DEPORTE Y RECREACION': '#7986CB',
    'HOTEL, MOTEL': '#F06292',
    'ESTACIONAMIENTO': '#00ACC1',
    'ADM PUBLICA Y DEFENSA': '#EF5350',
    'AGRICOLA': '#8BC34A',
    'S/I': '#D3D3D3'
}

In [9]:
# Crear un nuevo diccionario con las llaves de LBCS_translated y los valores de LBCS
LBCS_eng = {LBCS_translated[key]: value for key, value in LBCS.items() if key in LBCS_translated}

In [10]:
def read_grid_geojson(filename):    
    with open(filename, 'r') as file:
        grid_dict = json.load(file)
        return grid_dict

def extract_types_gdf(filename):
    grid_dict = read_grid_geojson(filename)
    types = pd.DataFrame.from_dict(grid_dict['GEOGRID']['properties']['types']).transpose()
    return types

grid_filename = '/app/data/input/grid_cityscope.geojson'

types = extract_types_gdf(grid_filename)

In [11]:
types.reset_index(inplace=True)

In [12]:
grid = gpd.read_parquet('/app/data/landuses_grid.parquet')
# Convertir arrays a strings
grid['height'] = grid['height'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
grid['color'] = grid['color'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

In [13]:
new_names = grid['Uso'].unique()

In [14]:
new_types = pd.DataFrame(columns=types.columns)
new_types.drop(columns=['id', 'index', 'color'], inplace=True)

In [15]:
new_types['index'] = new_names
new_types['name'] = new_names
new_types['description'] = ''
new_types['interactive'] = False

In [16]:
h = [0, 10, 20]
new_types['height'] = [h]*len(new_types)
new_types['height[0]'] = h[0]
new_types['height[1]'] = h[1]
new_types['height[2]'] = h[2]

In [17]:
new_types['NAICS'] = np.nan

In [18]:
new_types['name'] = new_types['name'].map(LBCS_translated)

In [19]:
# "LBCS": [
#     {
#         "proportion": 1,
#         "use": {
#             "1100": 1
#         }
#     }
# ],
# "NAICS": null,
for index, row in new_types.iterrows():
    c_type = row['name']
    lbcs_code = LBCS_eng[c_type]
    lbcs = [
        {
            "proportion": 1,
            "use": {
                lbcs_code: 1
            }

        }
    ]
    new_types.loc[index, 'LBCS'] = lbcs

In [20]:
color_dict = {
    'SITIO ERIAZO': '#E57373',
    'HABITACIONAL': '#81C784',
    'EDUCACION Y CULTURA': '#64B5F6',
    'BODEGA Y ALMACENAJE': '#F06292',
    'BIENES COMUNES': '#DCE775',
    'OTROS': '#4DD0E1',
    'INDUSTRIA': '#BA68C8',
    'SALUD': '#FFB74D',
    'CULTO': '#9575CD',
    'OFICINA': '#AED581',
    'COMERCIO': '#F48FB1',
    'TRANSPORTE Y TELEC': '#B0BEC5',
    'AREAS VERDES': '#4CAF50',
    'DEPORTE Y RECREACION': '#7986CB',
    'HOTEL, MOTEL': '#F06292',
    'ESTACIONAMIENTO': '#00ACC1',
    'ADM PUBLICA Y DEFENSA': '#EF5350',
    'AGRICOLA': '#8BC34A',
    'S/I': '#D3D3D3'  # Color gris claro para 'S/I'
}

new_types['color'] = new_types['index'].map(color_dict)

In [21]:
new_types['index'] = new_types['name']

In [22]:
new_types.reset_index(inplace=True)
new_types.rename(columns={'level_0': 'id'}, inplace=True)

In [23]:
new_types.set_index('index', inplace=True)

In [24]:
new_types_columns = ['id', 'name', 'description', 'height', 'height[0]', 'height[1]', 'height[2]', 'LBCS', 'NAICS', 'interactive', 'color']
new_types_export = new_types[new_types_columns]

In [25]:
output_filename = '/app/data/new_types.json'
new_types_export.to_json(output_filename, orient='index', index=True)

#### Unir todo para el grid exportable

In [26]:
grid_output['GEOGRID']['features'] = load_json('/app/data/GEOGRID.geojson')['features']
grid_output['GEOGRID']['properties']['types'] = load_json('/app/data/new_types.json')
grid_output['GEOGRIDDATA'] = load_json('/app/data/GEOGRIDDATA.json')

In [27]:
output_filename = '/app/data/testing.geojson'
with open(output_filename, 'w') as archivo:
    json.dump(grid_output, archivo)